<a href="https://colab.research.google.com/github/NeelamTharunKumar/Google_Colab/blob/main/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [ ]:

# Load Dataset
df = pd.read_csv("IMDB Dataset.csv")

# Keep 1.1K Positive & 1.1K Negative Reviews
df_positive = df[df['sentiment'] == 'positive'].sample(n=1100, random_state=1)
df_negative = df[df['sentiment'] == 'negative'].sample(n=1100, random_state=1)
df = pd.concat([df_positive, df_negative]).sample(frac=1, random_state=1)

df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})

# Train-Test Split
train_data = df.iloc[:2000]
test_data = df.iloc[2000:]


In [ ]:
X_train, y_train = train_data['review'].values, train_data['sentiment'].values
X_test, y_test = test_data['review'].values, test_data['sentiment'].values

In [ ]:
# Tokenization & Padding
vocab_size = 10000
max_length = 500
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

In [ ]:
# 1️⃣ Define the Model
model = Sequential()

model.add(SimpleRNN(100, activation='relu', input_shape=(max_length, 1)))  # RNN Layer with 100 units
model.add(Dense(250, activation='relu'))  # Fully connected layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# 2️⃣ Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 3️⃣ Print Model Summary
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 100)                 │          10,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 250)                 │          25,250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             251 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,701 (139.46 KB)

 Trainable params: 35,701 (139.46 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# 4️⃣ Reshape Input for RNN (RNN expects 3D inputs: batch_size, timesteps, features)
X_train_rnn = np.expand_dims(X_train_padded, axis=-1)
X_test_rnn = np.expand_dims(X_test_padded, axis=-1)

# 5️⃣ Train the Model
model.fit(X_train_rnn, y_train, validation_data=(X_test_rnn, y_test), epochs=10, batch_size=128, verbose=2)


Epoch 1/10
16/16 - 5s - 325ms/step - accuracy: 0.4935 - loss: 6.0466 - val_accuracy: 0.4950 - val_loss: 4.2903
Epoch 2/10
16/16 - 5s - 287ms/step - accuracy: 0.5100 - loss: 3.5094 - val_accuracy: 0.4850 - val_loss: 2.2928
Epoch 3/10
16/16 - 3s - 183ms/step - accuracy: 0.5140 - loss: 1.8070 - val_accuracy: 0.4750 - val_loss: 4.7747
Epoch 4/10
16/16 - 5s - 319ms/step - accuracy: 0.5185 - loss: 1.8363 - val_accuracy: 0.4600 - val_loss: 5.4777
Epoch 5/10
16/16 - 6s - 401ms/step - accuracy: 0.5155 - loss: 1.6005 - val_accuracy: 0.4950 - val_loss: 1.9714
Epoch 6/10
16/16 - 3s - 179ms/step - accuracy: 0.5205 - loss: 0.9410 - val_accuracy: 0.4800 - val_loss: 1.3256
Epoch 7/10
16/16 - 3s - 178ms/step - accuracy: 0.5210 - loss: 0.8972 - val_accuracy: 0.4800 - val_loss: 1.7557
Epoch 8/10
16/16 - 6s - 398ms/step - accuracy: 0.5205 - loss: 1.3019 - val_accuracy: 0.4950 - val_loss: 1.3341
Epoch 9/10
16/16 - 4s - 257ms/step - accuracy: 0.5185 - loss: 1.1347 - val_accuracy: 0.4850 - val_loss: 2.0821
E

In [ ]:
# 6️⃣ Evaluate the Model
scores = model.evaluate(X_test_rnn, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

# 7️⃣ Save the Model
model.save("sentiment_model_no_embedding.h5")

Accuracy: 47.50%


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

embedding_dim = 100
vocab_size = 10000
max_length = 500

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, max_length))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


print(model.summary())



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 500, 100)            │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 250)                 │      12,500,250 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             251 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,500,501 (51.50 MB)

 Trainable params: 13,500,501 (51.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# 7️⃣ Train the Model
model.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=5, batch_size=128, verbose=2)



Epoch 1/5
16/16 - 10s - 611ms/step - accuracy: 0.5130 - loss: 1.2200 - val_accuracy: 0.5750 - val_loss: 0.6855
Epoch 2/5
16/16 - 10s - 634ms/step - accuracy: 0.5795 - loss: 0.6875 - val_accuracy: 0.5100 - val_loss: 0.6919
Epoch 3/5
16/16 - 9s - 543ms/step - accuracy: 0.5590 - loss: 0.6797 - val_accuracy: 0.5500 - val_loss: 0.6956
Epoch 4/5
16/16 - 10s - 653ms/step - accuracy: 0.5825 - loss: 0.6527 - val_accuracy: 0.5300 - val_loss: 0.7226
Epoch 5/5
16/16 - 8s - 512ms/step - accuracy: 0.6105 - loss: 0.6392 - val_accuracy: 0.5350 - val_loss: 0.7179


In [ ]:

scores = model.evaluate(X_test_padded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

model.save("sentiment_model_with_embedding.h5")


Accuracy: 53.50%
